In [1]:
import keras.datasets.imdb

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(
    path="imdb.npz",
    num_words=None,
    skip_top=0,
    maxlen=None,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3
)

In [3]:
word_index = keras.datasets.imdb.get_word_index()
inverted_word_index = dict((i, word) for (word, i) in word_index.items())
decoded_sequence = " ".join(inverted_word_index[i] for i in x_train[0])

print(decoded_sequence)

the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room titillate it so heart shows to years of every never going villaronga help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but pratfalls to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other tricky in of seen over landed for anyone of gilmore's br show's to whether from than out themselves history he name half some br of 'n odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but whe

In [4]:
print(type(word_index.values()))
print(type(list(word_index.values())))

<class 'dict_values'>
<class 'list'>


In [5]:
#visualisation de l'equivalent dans le vocabulaire
decoded_sequence

"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room titillate it so heart shows to years of every never going villaronga help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but pratfalls to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other tricky in of seen over landed for anyone of gilmore's br show's to whether from than out themselves history he name half some br of 'n odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but wh

In [6]:
import numpy as np

# utilisation de numpy pour verifier que la separation des jeux a ete correctement effectue

print(type(x_train))
np.array(x_train).shape

<class 'numpy.ndarray'>


(25000,)

In [7]:
def get_total_length(x_train):
    res = 0
    for l in x_train:
        res += len(l)
    return res

In [36]:
import math
from tqdm import tqdm

def naive_bayes_classifier_train(x_train, y_train):
    """
    Classificateur Bayesien naif
    Inputs: le trainset du dataset imdb
    Output: TODO
    """
    # tuple valeur/etiquette dans chaque element de imbd
    ndoc = len(x_train)
    nbclasses = len(np.unique(y_train))
    logprior = [0] * nbclasses
    bigdoc = [[]] * nbclasses
    voca = list(keras.datasets.imdb.get_word_index().values())
    loglikelyhood = np.ndarray((len(voca), nbclasses))
    for c in range(nbclasses):
        nlogdoc = math.log(ndoc)
        # array numpy, donc masque
        c_class_train = x_train[y_train == c]
        nc = len(c_class_train)
        logprior[c] = nlogdoc / math.log(nc)
        #recuperation des cles sous formes numerisees
        occu_voca = np.zeros(len(voca))
        for docu in tqdm(c_class_train):
            occu_voca = np.add(occu_voca, np.histogram(docu, bins=np.arange(len(voca) + 1))[0])
        nb_word_total  = get_total_length(c_class_train)
        for index in tqdm(range(len(voca))):
            value = math.log((occu_voca[index] + 1) /
                                      (nb_word_total + 1))
            loglikelyhood[index, c] = value
    return logprior, loglikelyhood, voca

In [38]:
trained_classifier = naive_bayes_classifier_train(x_train, y_train)
print(trained_classifier)

100%|██████████| 88584/88584 [00:00<00:00, 916841.60it/s]

([1.073477326743371, 1.073477326743371], array([[-14.89674099, -14.9206144 ],
       [ -5.46317707,  -5.48705048],
       [-14.20359381, -14.9206144 ],
       ...,
       [-14.20359381, -14.9206144 ],
       [-14.20359381, -14.9206144 ],
       [-14.20359381, -14.22746722]]), [34701, 52006, 52007, 16816, 63951, 1408, 16115, 2345, 2289, 52008, 52009, 11307, 40830, 30568, 52010, 40831, 52011, 19313, 52012, 52013, 25242, 6746, 52014, 52015, 52016, 68804, 52017, 40833, 34702, 2338, 40834, 34703, 52018, 16817, 1636, 16818, 52019, 34704, 52020, 11585, 57766, 52021, 14129, 52023, 11038, 52025, 29114, 52027, 52125, 40835, 52028, 52130, 34706, 27631, 40836, 15492, 52030, 11926, 4010, 3230, 52031, 34707, 30585, 52033, 40837, 26338, 52034, 30569, 52035, 52036, 40839, 52037, 52038, 11927, 16819, 52039, 25243, 21905, 52040, 40840, 40841, 359, 5034, 7093, 21906, 23379, 52041, 52042, 18510, 30570, 9878, 40842, 52043, 52044, 52045, 40843, 34708, 25244, 7180, 52046, 40844, 11586, 20598, 52047, 11037, 5